## 1) Markdown practice

### 2.2.1. This is a level 3 heading

There are also single and double hashtag titles for level 1 and 2 - but I wanted to keep my table of contents for this tutorial clean and tidy :)

You can also type some text here.

This is how you do something in **bold** or _italics_

Then you can make a list of bullet points:

* 1, 2, 3
* A, B, C
* what about an indented bullet?
  + as easy as it gets!

1. What if you want numbered lists?
2. As easy as this!

We can also add images using a few methods as long as the file is in the right path!

<img alt="alternate text goes here" src="lets-do-this.jpg" width="60%" />

![You can add some alternate text here](doh.jpg)

Finally - we can add code chunks like so to have SQL syntax highlighting turned on:

```sql
SELECT * FROM some_table
```

Or you can have code in-line like this `SELECT * FROM some_table` but the syntax highlighting won't on for these in-line code snippets.

If you want to highlight something you can also use blockquotes:

> Something you really want to emphasise

You can also do page breaks using three underscores together like below to slightly separate sections of your document

---

Finally we can also include external links to websites like [https://www.google.com]

## 2) SELECT, ORDER BY practice

What is the name of the category with the highest category_id in the dvd_rentals.category table?

In [ ]:
SELECT
name,
category_id
FROM dvd_rentals.category
ORDER BY 2 DESC
LIMIT 1;

For the films with the longest length, what is the title of the “R” rated film with the lowest replacement_cost in dvd_rentals.film table?

In [ ]:
SELECT
title,
replacement_cost,
length,
rating
FROM dvd_rentals.film
ORDER BY 3 DESC,2
LIMIT 10;

Who was the manager of the store with the highest total_sales in the dvd_rentals.sales_by_store table?

In [ ]:
SELECT
manager,
sales_by_store
FROM dvd_rentals.sales_by_store
ORDER BY 2 DESC
LIMIT 1;

What is the postal_code of the city with the 5th highest city_id in the dvd_rentals.address table?

In [ ]:
SELECT
postal_code,
city_id
FROM dvd_rentals.address
ORDER BY 2 DESC
LIMIT 5;

## 3) Record Counts & Distinct Values

Which actor_id has the most number of unique film_id records in the dvd_rentals.film_actor table?


In [ ]:
SELECT
  actor_id,
  COUNT(*) AS frequency
FROM dvd_rentals.film_actor
GROUP BY actor_id
ORDER BY frequency DESC
LIMIT 5;

How many distinct fid values are there for the 3rd most common price value in the dvd_rentals.nicer_but_slower_film_list table?

In [ ]:
SELECT
  price,
  COUNT(DISTINCT(fid)) AS frequency
FROM dvd_rentals.nicer_but_slower_film_list
GROUP BY price
ORDER BY frequency DESC
LIMIT 5
OFFSET 2;

How many unique country_id values exist in the dvd_rentals.city table?


In [ ]:
SELECT
  COUNT(DISTINCT(country_id)) AS unique_country_id
FROM dvd_rentals.city
;

What percentage of overall total_sales does the Sports category make up in the dvd_rentals.sales_by_film_category table?

In [ ]:
SELECT 
category,
ROUND(100*total_sales::numeric/SUM(total_sales) OVER (), 2) AS percentage
FROM dvd_rentals.sales_by_film_category;

What percentage of unique fid values are in the Children category in the dvd_rentals.film_list table?


In [ ]:
SELECT 
  category,
  ROUND(100 * COUNT(DISTINCT fid)::NUMERIC/SUM(COUNT(fid)) OVER(), 2) AS percentage
FROM dvd_rentals.film_list
GROUP BY category;

### 4) Identifying Duplicate Data


Which id value has the most number of duplicate records in the health.user_logs table?

In [ ]:
WITH Freq_count AS (
SELECT 
 id,
 log_date,
 measure,
 measure_value,
 systolic,
 diastolic,
 COUNT (*) AS freq
FROM health.user_logs
GROUP BY 
 id,
 log_date,
 measure,
 measure_value,
 systolic,
 diastolic
)
SELECT 
 id,
 SUM(freq) AS numer_of_dublicates
FROM Freq_count
WHERE freq > 1
GROUP BY 
 id
ORDER BY numer_of_dublicates DESC
LIMIT 10;

Which log_date value had the most duplicate records after removing the max duplicate id value from question 1?

In [ ]:
WITH Freq_count AS (
SELECT 
    id,
    log_date,
    measure,
    measure_value,
    systolic,
    diastolic,
    COUNT (*) AS freq
FROM health.user_logs
WHERE id != '054250c692e07a9fa9e62e345231df4b54ff435d'
GROUP BY 
    id,
    log_date,
    measure,
    measure_value,
    systolic,
    diastolic
)
SELECT 
    log_date,
    SUM(freq) AS numer_of_dublicates
FROM Freq_count
WHERE freq > 1
GROUP BY 
    log_date
ORDER BY numer_of_dublicates DESC
LIMIT 10;

Which measure_value had the most occurences in the health.user_logs value when measure = 'weight'?

In [ ]:
SELECT 
    measure_value,
    COUNT(measure_value) AS numer_of_instances
FROM health.user_logs
WHERE measure = 'weight'
GROUP BY measure_value
ORDER BY numer_of_instances DESC
LIMIT 10;

How many single duplicated rows exist when measure = 'blood_pressure' in the health.user_logs? How about the total number of duplicate records in the same table?

In [ ]:
WITH health_data AS( 
SELECT 
    id, 
    log_date, 
    measure, 
    measure_value, 
    systolic, 
    diastolic, 
    COUNT(*) AS frequency 
FROM health.user_logs 
WHERE measure = 'blood_pressure' 
GROUP BY 
    id, 
    log_date, 
    measure, 
    measure_value, 
    systolic, 
    diastolic
HAVING COUNT(*) > 1
) 
SELECT 
    COUNT (frequency) AS Single_dublicated_records,
    SUM (frequency) AS Total_record_number
FROM health_data

What percentage of records measure_value = 0 when measure = 'blood_pressure' in the health.user_logs table? How many records are there also for this same condition?

In [ ]:
WITH health_data AS (
SELECT 
measure_value,
COUNT(measure_value) AS number_of_records,
SUM(COUNT(measure_value)) OVER () AS Total_records
FROM health.user_logs
WHERE measure = 'blood_pressure'
GROUP BY measure_value
)
SELECT 
*,
ROUND(100* number_of_records::NUMERIC/total_records, 2) AS percentage
FROM health_data
WHERE measure_value = 0


What percentage of records measure_value = 0 when measure = 'blood_pressure' in the health.user_logs table? How many records are there also for this same condition?

In [ ]:
WITH groupby_counts AS (
  SELECT
    id,
    log_date,
    measure,
    measure_value,
    systolic,
    diastolic,
    COUNT(*) AS frequency
  FROM
    health.user_logs
  GROUP BY
    id,
    log_date,
    measure,
    measure_value,
    systolic,
    diastolic
)
SELECT
  ROUND(
    100 * SUM(
      CASE
        WHEN frequency > 1 THEN frequency - 1
        ELSE 0
      END
    ) :: NUMERIC / SUM(frequency),
    2
  ) AS duplicate_percentage
FROM
  groupby_counts;

### 4) Descriptive statistics 

Ok - let’s try calculating all of the statistics we’ve covered so far on our real dataset to see what we can find out about our values.

In [1]:
SELECT
'weight' AS measure,
ROUND(MIN(measure_value), 2) AS minimum,
ROUND(MAX(measure_value), 2) AS maximum,
ROUND(AVG(measure_value), 2) AS mean,
ROUND(CAST(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY measure_value) AS NUMERIC), 2) AS median, 
ROUND(MODE() WITHIN GROUP (ORDER BY measure_value), 2) AS mode,
ROUND(VARIANCE(measure_value), 2) AS variance,
ROUND(STDDEV(measure_value), 2) AS Standard_deviation

FROM health.user_logs
WHERE measure = 'weight'

SyntaxError: invalid syntax (1209565813.py, line 2)

What is the average, median and mode values of blood glucose values to 2 decimal places?

In [ ]:
SELECT
  'blood glucose' AS measure,
  ROUND(AVG(measure_value), 2) AS mean,
  ROUND(
    CAST(
      PERCENTILE_CONT(0.5) WITHIN GROUP (
        ORDER BY
          measure_value
      ) AS NUMERIC
    ),
    2
  ) AS median,
  ROUND(
    MODE() WITHIN GROUP (
      ORDER BY
        measure_value
    ),
    2
  ) AS mode
FROM
  health.user_logs
WHERE
  measure = 'blood_glucose';

What is the most frequently occuring measure_value value for all blood glucose measurements?

In [ ]:
SELECT
  measure_value,
  COUNT(*) AS frequency
FROM
  health.user_logs
WHERE
  measure = 'blood_glucose'
GROUP BY
  measure_value
ORDER BY
  frequency DESC
LIMIT
  10;

Calculate the 2 Pearson Coefficient of Skewness for blood glucose measures given the following formulas:

In [ ]:
WITH data_preprocess AS (
  SELECT
    'blood glucose' AS measure,
    ROUND(AVG(measure_value), 2) AS mean,
    ROUND(
      CAST(
        PERCENTILE_CONT(0.5) WITHIN GROUP (
          ORDER BY
            measure_value
        ) AS NUMERIC
      ),
      2
    ) AS median,
    ROUND(
      MODE() WITHIN GROUP (
        ORDER BY
          measure_value
      ),
      2
    ) AS mode,
    ROUND(STDDEV(measure_value), 2) AS Standard_deviation
  FROM
    health.user_logs
  WHERE
    measure = 'blood_glucose'
)
SELECT
  measure,
  ROUND((mean - mode) / Standard_deviation, 4) AS Coefficient1,
  ROUND(3* (mean - median)/ Standard_deviation, 4) AS Coefficient2
FROM
  data_preprocess

Check each value position in percentile

In [ ]:
WITH Percentile_values AS(
  SELECT
    measure_value,
    NTILE(100) OVER (
      ORDER BY
        measure_value
    ) AS percentile
  FROM
    health.user_logs
  WHERE
    measure = 'weight'
    AND measure_value > 0
    AND measure_value < 201
)
SELECT
  measure_value,
  ROW_NUMBER() OVER (
    ORDER BY
      measure_value
  ) as row_number_order,
  RANK() OVER (
    ORDER BY
      measure_value
  ) as rank_order,
  DENSE_RANK() OVER (
    ORDER BY
      measure_value
  ) as dense_rank_order,
  percentile
FROM
  Percentile_values
WHERE
  percentile = 1
ORDER BY
  percentile

Clean Percentile calculations

In [ ]:
DROP TABLE IF EXISTS clean_weight_logs;
CREATE TEMP TABLE clean_weight_logs AS (
  SELECT *
  FROM health.user_logs
  WHERE measure = 'weight'
    AND measure_value > 0
    AND measure_value < 201
);
SELECT
  ROUND(MIN(measure_value), 2) AS minimum_value,
  ROUND(MAX(measure_value), 2) AS maximum_value,
  ROUND(AVG(measure_value), 2) AS mean_value,
  ROUND(
    -- this function actually returns a float which is incompatible with ROUND!
    -- we use this cast function to convert the output type to NUMERIC
    CAST(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY measure_value) AS NUMERIC),
    2
  ) AS median_value,
  ROUND(
    MODE() WITHIN GROUP (ORDER BY measure_value),
    2
  ) AS mode_value,
  ROUND(STDDEV(measure_value), 2) AS standard_deviation,
  ROUND(VARIANCE(measure_value), 2) AS variance_value
FROM clean_weight_logs;

WITH percentile_values AS (
  SELECT
    measure_value,
    NTILE(100) OVER (
      ORDER BY
        measure_value
    ) AS percentile
  FROM clean_weight_logs
)
SELECT
  percentile,
  MIN(measure_value) AS floor_value,
  MAX(measure_value) AS ceiling_value,
  COUNT(*) AS percentile_counts
FROM percentile_values
GROUP BY percentile
ORDER BY percentile;

We’ve just received an urgent request from the General Manager of Analytics at Health Co requesting our assistance with their analysis of the health.user_logs dataset.

The Health Co analytics team have shared with us their SQL script - they unfortunately ran into a few bugs that they couldn’t fix!

We’ve been asked to quickly debug their SQL script and use the resulting query outputs to quickly answer a few questions that the GM has requested for a board meeting about their active users.

In [ ]:
-- 1. How many unique users exist in the logs dataset?
SELECT
  COUNT(DISTINCT id)
FROM health.user_logs;


-- for questions 2-8 we created a temporary table
DROP TABLE IF EXISTS user_measure_count;
CREATE TEMP TABLE user_measure_count AS(
SELECT
    id,
    COUNT(*) AS measure_count,
    COUNT(DISTINCT measure) as unique_measures
  FROM health.user_logs
  GROUP BY 1); 


-- 2. How many total measurements do we have per user on average?
SELECT
  ROUND(AVG(measure_count), 2)
FROM user_measure_count;

-- 3. What about the median number of measurements per user?
SELECT
  ROUND(CAST(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY measure_count) AS NUMERIC), 2) AS median
FROM user_measure_count;

-- 4. How many users have 3 or more measurements?
SELECT
  COUNT(*)
FROM user_measure_count
WHERE measure_count >= 3;

-- 5. How many users have 1,000 or more measurements?
SELECT
  COUNT(*)
FROM user_measure_count
WHERE measure_count >= 1000;

-- 6. Have logged blood glucose measurements?
SELECT
  DISTINCT measure
FROM health.user_logs;

SELECT
  measure_value
FROM health.user_logs
WHERE measure = 'blood_glucose'
LIMIT 10;


-- 7. Have at least 2 types of measurements?
SELECT
  COUNT(*)
FROM user_measure_count
WHERE unique_measures >= 2;

-- 8. Have all 3 measures - blood glucose, weight and blood pressure?
SELECT
  COUNT(*)
FROM user_measure_count
WHERE unique_measures = 3;

-- 9.  What is the median systolic/diastolic blood pressure values?
SELECT
  PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY systolic) AS median_systolic,
  PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY diastolic) AS median_diastolic
FROM health.user_logs
WHERE measure = 'blood_pressure';
